In [1]:
pwd

'C:\\Projects\\Structures\\develop'

In [2]:
import pandas as pd
import numpy as np
import pyodbc
import sys
import json
import urllib
import sqlalchemy
import os
import shapely
from datetime import datetime

sys.path.append('../')
#from IPM_Shared_Code_public.Python.geo_functions import read_geosql
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.email_functions import get_contacts, read_template, send_email
from geosupport import *
from get_address_points import *

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
parksgis = config['srv']['parksgis']
data_parks_server = config['srv']['data_parks']
structuresdb = config['db']['structuresdb']
portal = config['url']['portal']
structures_url = config['url']['structures']
geosupport_url = config['url']['geosupport']

In [4]:
geo_key = config['keys']['geosupport_key']
geo_ip = config['keys']['geosupport_ip']

In [5]:
# params = urllib.parse.quote_plus(r'Driver=' + driver + ';Server=' + server +
#                                  ';Database=' + structuresdb +
#                                  ';Trusted_Connection=Yes;')
# engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
# connection = engine.connect()

In [6]:
b = pd.read_csv('structs_top5.csv')
b

,Unnamed: 0,fid,objectid,parks_id,bin,bbl,doitt_id,ground_elevation,height_roof,construction_year,alteration_year,demolition_year,n_doitt_ids,doitt_source,overlap_except,unexpected_change,parks_overlap,shape,row_hash
0,0,234,5139,B001-BLG0989,3400033.0,3025650014,1157878.0,15.0,14.910000,1940.0,NaN,NaN,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x00...,b'v+\xdd\x9ek\x9e\xc3Jq\x0cP[\xeeRm$\xe2\x1d\x...
1,2,1204,6494,B007-BLG1117,3170079.0,3064900024,609048.0,21.0,15.060000,1900.0,NaN,NaN,1,current,True,False,False,"b""\xd7\x08\x00\x00\x01\x04\t\x00\x00\x00\x80\x...",b'^\xe8f\x8c\x9a\x8c\xe8l\x91;\x16\x9aP@\x8f\x...
2,3,1979,6185,B007-BLG1118,3340691.0,3064900024,921467.0,10.0,21.000000,1900.0,NaN,NaN,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\n\x00\x00\x00\x80\x...,b'#\\Y\x1bS\xe94\x17\x02@\xfbqd}B\xdc\x05\xa3\...
3,4,1320,6184,B007-BLG1119,3423964.0,3064900024,1272109.0,13.0,12.049366,NaN,NaN,NaN,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x07\x00\x00\x00\x00...,b'\x14x\xa4\x9c\x9fR8\xfc\x7f\x80aXP\xeb\xfb\x...


In [7]:
# sql = 'select top 5 * from structuresdb.dbo.tbl_parks_structures'

In [8]:
# b = pd.read_sql(con=engine, sql=sql)

In [10]:
structs = b[~pd.isnull(b['bin'])]

In [11]:
final_df = master_geosupport_func2(structs)

In [12]:
final_df

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
0,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
2,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
3,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
4,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


# Dan, you can ignore cells below this one for now

In [10]:
# structs.to_csv(r'structs_all.csv')

In [11]:
# structs.to_csv(r'structs_top5.csv')

In [12]:
bins = list(structs.bin.values)

In [13]:
# bins = list(b[b['bin'].notnull()]['bin'])

In [14]:
# bins

In [73]:
structs

,Unnamed: 0,fid,objectid,parks_id,bin,bbl,doitt_id,ground_elevation,height_roof,construction_year,alteration_year,demolition_year,n_doitt_ids,doitt_source,overlap_except,unexpected_change,parks_overlap,shape,row_hash
0,0,234,5139,B001-BLG0989,3400033.0,3025650014,1157878.0,15.0,14.910000,1940.0,NaN,NaN,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x05\x00\x00\x00\x00...,b'v+\xdd\x9ek\x9e\xc3Jq\x0cP[\xeeRm$\xe2\x1d\x...
1,2,1204,6494,B007-BLG1117,3170079.0,3064900024,609048.0,21.0,15.060000,1900.0,NaN,NaN,1,current,True,False,False,"b""\xd7\x08\x00\x00\x01\x04\t\x00\x00\x00\x80\x...",b'^\xe8f\x8c\x9a\x8c\xe8l\x91;\x16\x9aP@\x8f\x...
2,3,1979,6185,B007-BLG1118,3340691.0,3064900024,921467.0,10.0,21.000000,1900.0,NaN,NaN,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\n\x00\x00\x00\x80\x...,b'#\\Y\x1bS\xe94\x17\x02@\xfbqd}B\xdc\x05\xa3\...
3,4,1320,6184,B007-BLG1119,3423964.0,3064900024,1272109.0,13.0,12.049366,NaN,NaN,NaN,1,current,True,False,False,b'\xd7\x08\x00\x00\x01\x04\x07\x00\x00\x00\x00...,b'\x14x\xa4\x9c\x9fR8\xfc\x7f\x80aXP\xeb\xfb\x...


In [74]:
# %%timeit
get_ap_output = structs.apply(lambda row: get_address_point2(row['bin'],
              geom_col = 'the_geom'),axis =1)

In [75]:
aps_gdf = gpd.GeoDataFrame(flat_list(get_ap_output), geometry = 'the_geom',crs = 'epsg:4326').fillna(np.nan)
aps_gdf.head()

,address_id,the_geom,bin,h_no,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,physicalid,post_type,full_stree,modified
0,5168623,POINT (-73.95761 40.72873),3400033,81,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,64739,ST,FRANKLIN ST,NaN
1,3217153,POINT (-73.99996 40.59789),3170079,2134,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,42457,AVE,CROPSEY AVE,NaN
2,3217154,POINT (-74.00143 40.59508),3340691,9000,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,51350,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z
3,NaN,None,3.42396e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
aps = aps_gdf[~pd.isnull(aps_gdf['the_geom'])].to_crs('epsg:2263').copy()
aps.head()

,address_id,the_geom,bin,h_no,hyphen_typ,side_of_st,borocode,zipcode,created,st_name,physicalid,post_type,full_stree,modified
0,5168623,POINT (995998.149 204776.450),3400033,81,N,1,3,11222,2015-12-15T00:00:00.000Z,FRANKLIN,64739,ST,FRANKLIN ST,NaN
1,3217153,POINT (984261.245 157105.052),3170079,2134,N,2,3,11214,2009-02-13T00:00:00.000Z,CROPSEY,42457,AVE,CROPSEY AVE,NaN
2,3217154,POINT (983853.114 156079.787),3340691,9000,N,2,3,11214,2009-02-13T00:00:00.000Z,BAY,51350,PKWY,BAY PKWY,2018-07-26T00:00:00.000Z


In [19]:
# get_ap_output.values

In [20]:
# try_flat1 = flat_list(get_ap_output)#.loc[0]#.keys()
# try_flat1

In [21]:
# pd.DataFrame(try_flat1)

In [22]:
# aps, failed_bins = get_address_point(bins, geom_col = 'the_geom')

In [23]:
# type(aps)

In [24]:
# aps.head()

In [25]:
# failed_bins

In [26]:
# aps.bin.values

In [27]:
grc_err = ['01/F', '20', '21', '22', '23']
out_keys = [
    'AddressRangeList', 'out_bbl', 'out_TPAD_bin', 'out_TPAD_bin_status',
    'out_TPAD_conflict_flag', 'out_error_message', 'out_grc',
    'out_sanborn_boro', 'in_bin', 'out_boro_name1']

In [28]:
# for abin in bins
# bn = funcbn(bn = bins[2], out_keys = out_keys, grc_err = grc_err, api_key = geo_key, ip = geo_ip)

In [29]:
# structs['bin']

In [77]:
funcBN_outputs = structs.apply(lambda row: funcbn(row['bin'],
              out_keys=out_keys,
              grc_err=grc_err,
              api_key=geo_key,
              ip=geo_ip),axis =1)

In [31]:
# funcBN_outputs

In [78]:
t = flat_list(funcBN_outputs)
# t

In [79]:
strip_vals(t)

In [34]:
# t

In [80]:
BN_out_df = pd.DataFrame(t)
BN_out_df.head()

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin,out_boro_name1
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033,BROOKLYN
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079,BROOKLYN
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691,BROOKLYN
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964,BROOKLYN


In [36]:
# pd.DataFrame(strip_vals(flat_list(funcBN_outputs.values)))

In [37]:
# bn

In [38]:
# a = aps.iloc[0][['borocode', 'full_stree']]

In [39]:
# aps.values

In [81]:
func1N_outputs = aps.apply(lambda row: func1n(borough=row['borocode'],
                             streetname=row['full_stree'],
                             api_key=geo_key,
                             ip=geo_ip),axis =1)

In [82]:
list_1N = flat_list(func1N_outputs)

In [42]:
# list_1N

In [83]:
strip_vals(list_1N)

In [44]:
# list_1N

In [84]:
func1N_out_df = pd.DataFrame(list_1N)

In [85]:
aps_1N_df = aps.join(func1N_out_df)

In [47]:
# aps

In [48]:
# func1N_out_df

In [86]:
BN_out_df['func_name'] = 'BN'

In [87]:
aps_1N_df['func_name'] = '1N'

In [51]:
# func1N_out_df['func_name'] = '1N'

In [52]:
# aps_1N_df.columns.values

In [53]:
# BN_out_df.columns.values

In [54]:
# BN_out_df[['bin','high_address_number','low_address_number','street_name','out_boro_name1']]

In [88]:
aps_1N_df['high_address_number'] = aps_1N_df['h_no']

In [89]:

aps_1N_df['low_address_number'] = aps_1N_df['h_no']

In [90]:
aps_1N_df.rename(columns = {'out_stname1':'street_name'},inplace = True)

In [91]:
cols_to_match = ['bin','high_address_number','low_address_number','street_name','out_boro_name1']

In [92]:
# cols_to_match+['address_id','func_name']

In [60]:
# aps_1N_df

In [117]:
df1 = aps_1N_df[cols_to_match+['address_id','func_name']]
df1

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1N
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1N
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N


In [118]:
df2 = BN_out_df#[cols_to_match+['func_name','type_meaning']]
df2

,b7sc,bin,high_address_number,low_address_number,street_name,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin,out_boro_name1,func_name
0,34203001,3400033,81,81,FRANKLIN STREET,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033,BROOKLYN,BN
1,33063001,3170079,2134,2134,CROPSEY AVENUE,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079,BROOKLYN,BN
2,31733001,3340691,9000,8922,BAY PARKWAY,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691,BROOKLYN,BN
3,31049501,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964,BROOKLYN,BN


In [119]:
combined_deduped = pd.concat([df1,df2]).drop_duplicates(cols_to_match,keep='last')
combined_deduped

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,NaN,BN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,NaN,BN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
2,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,BN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
3,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,BN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


In [120]:
combined = pd.concat([df1,df2])
combined

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,NaN,BN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,NaN,BN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
2,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,BN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
3,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,BN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


In [121]:
df_w_ap_ids = pd.concat(g for _, g in combined.groupby(cols_to_match) if len(g) > 1).drop_duplicates(cols_to_match,keep='first')

In [122]:
df_new = combined_deduped.merge(df_w_ap_ids,how = 'left',suffixes=('', '_y'), 
                       left_on = cols_to_match, right_on=cols_to_match)
# df_new.drop(df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)

In [123]:
df_new.address_id.fillna(df_new.address_id_y, inplace=True)

In [124]:
df_new.drop(df_new.filter(regex='_y$').columns.tolist(),axis=1, inplace=True)
df_new

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name,b7sc,tpad_bin_status,type,type_meaning,out_bbl,out_TPAD_bin,out_TPAD_bin_status,out_TPAD_conflict_flag,out_error_message,out_grc,out_sanborn_boro,in_bin
0,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,BN,34203001,N/A,,Ordinary Address Range,3025650014,N/A,N/A,N/A,,00,3,3400033
2,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,BN,33063001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3170079
3,3340691,9000,8922,BAY PARKWAY,BROOKLYN,NaN,BN,31733001,N/A,,Ordinary Address Range,3064900024,N/A,N/A,N/A,,00,3,3340691
4,3423964,,,BELT PARKWAY EXIT 5 EASTBOUND,BROOKLYN,NaN,BN,31049501,N/A,B,Non-Addressable Unnamed Building,3064900024,N/A,N/A,N/A,,00,3,3423964


In [69]:
combined_deduped.groupby('bin').count()[]

,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name
bin,,,,,,
0.0,0,0,0,1,0,1
1000000.0,0,0,0,1,0,1
1003210.0,0,0,0,1,0,1
1088300.0,0,0,0,1,0,1
1090070.0,0,0,0,1,0,1
...,...,...,...,...,...,...
5169770,2,2,1,1,1,2
5169802,1,1,1,1,0,1
5169908,2,2,2,2,0,2


In [70]:
combined_deduped[combined_deduped['high_address_number']==combined_deduped['low_address_number']]

,bin,high_address_number,low_address_number,street_name,out_boro_name1,address_id,func_name
0,3400033,81,81,FRANKLIN STREET,BROOKLYN,5168623,1N
1,3170079,2134,2134,CROPSEY AVENUE,BROOKLYN,3217153,1N
2,3340691,9000,9000,BAY PARKWAY,BROOKLYN,3217154,1N
5,3081931,694,694,WASHINGTON AVENUE,BROOKLYN,3109361,1N
6,0,512,512,WASHINGTON AVENUE,BROOKLYN,5161024,1N
...,...,...,...,...,...,...,...
3737,2086438,,,5 BOROUGH SHOPS,BRONX,NaN,BN
3738,2086438,850,850,EAST 138 STREET,BRONX,NaN,BN
3739,2086438,231,231,WALNUT AVENUE,BRONX,NaN,BN
3740,2086438,,,EAST 137 STREET,BRONX,NaN,BN


In [64]:
# func1n(borough = a['borocode'], streetname = a['full_stree'], api_key = geo_key, ip = geo_ip)

In [65]:
# df = master_geosupport_func(bins)

In [66]:
# df

In [67]:
# failed_bins

In [68]:
# aps